In [8]:
#---------Clean up stripyness from CANDElS_info.txt--------

#Imported some stuff. 
import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table, join

#1. You should remove the following galaxies:
    #GDS: s24 = 0.020 mJy
    #UDS: s24 = 0.030 mJy
    #COS: s24 = 0.023 mJy 
    #EGS: s24 = 0.050 mJy

candels = ascii.read('CANDELS_info.txt')

mjy = {'GDS':0.020, 'UDS':0.030, 'COS':0.023, 'EGS':0.050}
candels['del'] = ['no']

def fluxlim(row):
    for k, v in mjy.items():
        #k = field, v = value
        if candels[row]['field'] == k and candels[row]['s24(mJy)'] == v: 
            candels[row]['del'] = 'yes'
    
for i in range(len(candels['s24(mJy)'])): 
    fluxlim(i)

candels.sort(['del','field'])

candels = candels[0:4778]
candels.remove_column('del')

#2. Remove fagn_mir = 0 galaxies.
candels.sort(['fagn_mir','field'])
candels = candels[1028:len(candels['fagn_mir'])]
candels.sort(['field','ID'])

nlst = ['fagn_tot','lagn','lsf','sfr','lx','lbol','BHAR']
for i in range(len(nlst)): 
    candels[nlst[i]] = [0.0]
    

#Calculate SFR
def sfr(i): 
    #fagn_tot from fagn_mir; no units
    candels[i]['fagn_tot'] = 0.66 * (candels[i]['fagn_mir'] ** 2) - 0.035 * candels[i]['fagn_mir']
    
    #lagn from fagn_tot; log Lsun 
    candels[i]['lagn'] = np.log10(candels[i]['fagn_tot']) + candels[i]['lir']

    #lsf from lagn; log Lsun
    candels[i]['lsf'] = np.log10(10 ** candels[i]['lir'] - 10 ** candels[i]['lagn']) 
   
    #lsf into ergs/s 
    candels[i]['lsf'] = candels[i]['lsf'] + np.log10(3.826) + 33
    
    #sfr from lsf; log Msun/yr
    candels[i]['sfr'] = np.log10(4.5) -44 + candels[i]['lsf']
    
for i in range(len(candels['sfr'])): 
    sfr(i)

#Calculate BHAR
def bhar(i): 
    #lagn to log ergs/s
    candels[i]['lagn'] = candels[i]['lagn'] + np.log10(3.826) + 33
    
    #lx from lagn; log ergs/s
    candels[i]['lx'] = 28.629 +  0.339 * candels[i]['lagn']
    
    #lbol from lx; log ergs/s 
    candels[i]['lbol'] = -9.25 + 1.25 * candels[i]['lx']
    
    #BHAR from lbol; log Msun/yr 
    candels[i]['BHAR'] = (9 * 10 ** candels[i]['lbol'])/(29979245800 ** 2)
    if candels[i]['BHAR'] > 0: 
        candels[i]['BHAR'] = np.log10(candels[i]['BHAR']) + np.log10(1.58649268) - 26
    else: 
        candels[i]['BHAR'] = 0.0
        
for i in range(len(candels['BHAR'])):
    bhar(i)

#Open FITS files
galdict = {0:'cos', 1:'gds', 2:'egs', 3:'uds'}
merged = []
data = []
for k, v in galdict.items():
    merged.append(fits.open(v + '_merged_v1.1.fits', memmap=True))
    data.append(Table(merged[k][1].data))
    data[k]['field'] = [v.upper()]


#Joining the FITS with candels 
def ftbl(table): 
    return join(table['field','ID','M_med', 'zbest','PETRO_RADIUS'], candels, keys = ['field','ID'])

tbls = []
for i in range(4): 
    tbls.append(ftbl(data[i]))

for i in range(len(tbls[3]['M_med'])): 
    if np.log10(tbls[3][i]['M_med']) > 0:
        tbls[3][i]['M_med'] = np.log10(tbls[3][i]['M_med'])
    else: 
        tbls[3][i]['M_med'] = 0

def jtbl(t1, t2): 
    return join(t1, t2,join_type='outer')

candels = jtbl(jtbl(jtbl(tbls[0],tbls[1]),tbls[2]),tbls[3]) 

candels.sort('zbest')
candels = candels[798:3567]

candels.write('CANDELS_z.txt', format='ascii.latex', overwrite=True)
candels

/Users/vsong/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in log10


field,ID,M_med,zbest,PETRO_RADIUS,fagn_mir,lir,s24(mJy),fagn_tot,lagn,lsf,sfr,lx,lbol,BHAR
str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
COS,1855,10.4292,0.75,5.28,0.2,10.65,0.122,0.0194,42.5205466956,44.2242368546,0.877449368377,43.0434653298,44.5543316623,-1.24462916152
EGS,10811,8.8339,0.7508,5.28,0.3,10.81,0.031,0.0489,43.0820538248,44.3709711474,1.02418366114,43.2338162466,44.7922703083,-1.00669051552
UDS,6775,10.372912003,0.7517,3.96,0.05,10.75,0.108,-0.0001,nan,nan,nan,nan,nan,0.0
COS,10454,8.49693,0.752,4.62,0.2,10.22,0.035,0.0194,42.0905466956,43.7942368546,0.447449368377,42.8976953298,44.3721191623,-1.42684166152
UDS,24197,10.1760912591,0.753,3.5,0.1,10.68,0.136,0.0031,41.7541066595,44.2613965617,0.914609075465,42.7836421576,44.229552697,-1.56940812682
COS,14797,9.20137,0.754,5.28,0.2,10.79,0.07,0.0194,42.6605466956,44.3642368546,1.01744936838,43.0909253298,44.6136566623,-1.18530416152
UDS,23927,10.0530784435,0.7583,5.2800002,0.1,10.67,0.116,0.0031,41.7441066595,44.2513965617,0.904609075465,42.7802521576,44.225315197,-1.57364562682
EGS,17077,10.1782,0.759,3.5,0.1,11.23,0.312,0.0031,42.3041066595,44.8113965617,1.46460907546,42.9700921576,44.462615197,-1.33634562682
UDS,15468,9.54530711647,0.759,4.6199999,0.1,10.36,0.057,0.0031,41.4341066595,43.9413965617,0.594609075465,42.6751621576,44.093952697,-1.70500812682


In [10]:
#----------Splitting to graph by field------

#Imported some stuff. 
import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table, join

t = ascii.read('CANDELS_z.txt')
t.sort(['field', 'zbest'])


fbin = [0,650,1665,2081,len(t['field'])]
fields = ['COS','EGS','GDS','UDS']
field_t = [None, None, None, None]
        
for i in range(4): 
    field_t[i] = t[fbin[i]:fbin[i+1]] 
    field_t[i].sort('zbest')
    field_t[i].write(fields[i]+'_z.txt', format='ascii.latex', overwrite=True)

field_t[0]

field,ID,M_med,zbest,PETRO_RADIUS,fagn_mir,lir,s24(mJy),fagn_tot,lagn,lsf,sfr,lx,lbol,BHAR
str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
COS,1855,10.4292,0.75,5.28,0.2,10.65,0.122,0.0194,42.5205466956,44.2242368546,0.877449368377,43.0434653298,44.5543316623,-1.24462916152
COS,10454,8.49693,0.752,4.62,0.2,10.22,0.035,0.0194,42.0905466956,43.7942368546,0.447449368377,42.8976953298,44.3721191623,-1.42684166152
COS,14797,9.20137,0.754,5.28,0.2,10.79,0.07,0.0194,42.6605466956,44.3642368546,1.01744936838,43.0909253298,44.6136566623,-1.18530416152
COS,2500,10.4122,0.761,5.28,0.2,11.18,0.23,0.0194,43.0505466956,44.7542368546,1.40744936838,43.2231353298,44.7789191623,-1.02004166152
COS,543,10.5346,0.788,4.62,0.3,10.65,0.151,0.0489,42.9220538248,44.2109711474,0.864183661141,43.1795762466,44.7244703083,-1.07449051552
COS,4869,9.77306,0.79,5.94,0.5,10.57,0.098,0.1475,43.321536986,44.0834393534,0.736651867131,43.3150010383,44.8937512978,-0.905209525969
COS,27642,9.66947,0.79,3.96,0.1,10.54,0.062,0.0031,41.6141066595,44.1213965617,0.774609075465,42.7361821576,44.170227697,-1.62873312682
COS,8142,10.3407,0.79,4.62,0.3,10.74,0.26,0.0489,43.0120538248,44.3009711474,0.954183661141,43.2100862466,44.7626078083,-1.03635301552
COS,24980,10.4451,0.7926,3.5,0.1,10.77,0.066,0.0031,41.8441066595,44.3513965617,1.00460907546,42.8141521576,44.267690197,-1.53127062682


In [1]:
#--------Making GDS morphology table------ 
#Imported some stuff. 
import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table, join

#new table: ID, RA, Dec, SFR, BHAR, and Mstar
t = ascii.read('gds.txt')
col_names = ['#','ID','RA','Dec','redshift','redshift_err','zbest_candels',
            'gfit_n_h','SD_reliable','Irr_reliable','Compact_reliable',
            's14_RA','s14_Dec','s14_M_med',
             'visualHCPG_f_spheroid','visualHCPG_f_disk','visualHCPG_f_irr', 'visualHCPG_f_ps']

for i in range(len(t[0])): 
    t.rename_column('col' + str(i+1), col_names[i])

#for z 0-0.5 slice 0:4321
t['field'] = ['GDS']
t = t['field','ID','zbest_candels',
            'gfit_n_h','SD_reliable','Irr_reliable','Compact_reliable',
            's14_RA','s14_Dec',
             'visualHCPG_f_spheroid','visualHCPG_f_disk','visualHCPG_f_irr', 'visualHCPG_f_ps']

t

s = ascii.read('GDS_z.txt')

#s.sort('field')

j = join(t, s, keys = ['ID'])

j.write('gds_allz.txt', format='ascii.latex', overwrite=True)
j

field_1,ID,zbest_candels,gfit_n_h,SD_reliable,Irr_reliable,Compact_reliable,s14_RA,s14_Dec,visualHCPG_f_spheroid,visualHCPG_f_disk,visualHCPG_f_irr,visualHCPG_f_ps,field_2,M_med,zbest,PETRO_RADIUS,fagn_mir,lir,s24(mJy),fagn_tot,lagn,lsf,sfr,lx,lbol,BHAR
str3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
GDS,196,0.94,1.4,-99.0,-99.0,-99.0,53.14,-27.94,0.0,0.0,0.0,0.0,GDS,10.34,2.117,7.92,0.2,11.13,0.059,0.0194,43.0005466956,44.7042368546,1.35744936838,43.2061853298,44.7577316623,-1.04122916152
GDS,267,1.09,0.96,0.0,0.33,0.0,53.11,-27.94,0.1,0.87,0.29,0.0,GDS,9.95,1.083,4.62,0.1,10.78,0.072,0.0031,41.8541066595,44.3613965617,1.01460907546,42.8175421576,44.271927697,-1.52703312682
GDS,1033,1.8,0.2,0.25,0.38,0.0,53.12,-27.92,0.11,0.72,0.5,0.0,GDS,10.03,1.798,5.28,0.2,11.15,0.071,0.0194,43.0205466956,44.7242368546,1.37744936838,43.2129653298,44.7662066623,-1.03275416152
GDS,1191,1.67,0.88,0.0,0.62,0.0,53.11,-27.91,0.09,0.6,0.66,0.0,GDS,10.15,1.669,5.28,0.2,11.27,0.078,0.0194,43.1405466956,44.8442368546,1.49744936838,43.2536453298,44.8170566623,-0.981904161519
GDS,1194,2.12,0.72,0.0,0.25,0.0,53.17,-27.91,0.04,0.8,0.28,0.0,GDS,10.09,2.147,5.94,0.2,11.04,0.053,0.0194,42.9105466956,44.6142368546,1.26744936838,43.1756753298,44.7195941623,-1.07936666152
GDS,1239,2.0,0.91,0.0,0.38,0.0,53.18,-27.91,0.0,0.91,0.19,0.03,GDS,10.34,1.996,6.6,0.2,11.16,0.075,0.0194,43.0305466956,44.7342368546,1.38744936838,43.2163553298,44.7704441623,-1.02851666152
GDS,1379,1.11,1.73,0.92,0.0,0.25,53.22,-27.91,0.89,0.46,0.0,0.11,GDS,9.43,1.071,3.96,0.06,10.27,0.025,0.000276,40.2936540478,43.8526250839,0.505837597645,42.2885487222,43.6106859027,-2.18827492105
GDS,1404,1.1,4.41,1.0,0.62,0.0,53.18,-27.91,0.74,0.0,0.92,0.01,GDS,9.47,1.1043,4.62,0.1,10.5,0.038,0.0031,41.5741066595,44.0813965617,0.734609075465,42.7226221576,44.153277697,-1.64568312682
GDS,1446,1.31,0.6,0.0,0.0,0.0,53.19,-27.91,0.01,0.99,0.07,0.01,GDS,9.83,1.315,4.62,0.2,10.83,0.032,0.0194,42.7005466956,44.4042368546,1.05744936838,43.1044853298,44.6306066623,-1.16835416152
